In [2]:
import os
import glob
import pandas as pd
import calendar
import math
import itertools
import datetime
import win32com.client as win32

In [2]:
#Adjusting all the files extensions and changing to csv

path = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Forecasts\\'


# Specify the folder path

# Create a connection to Excel
excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = False

#Iterate over folders
#for item in ["GDP", "HCIP", "IndProd", "Unemployment"]:
for item in ["IndProd2"]:
    
    folder_path = os.path.join(path, item)
    
    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xls') or filename.endswith('.xlsx'):  # Process only Excel files
            # Construct the file paths
            excel_file_path = os.path.join(folder_path, filename)
            base_name = os.path.splitext(filename)[0]
            csv_file_path = os.path.join(folder_path, base_name + '.csv')

            # Open the Excel file
            workbook = excel.Workbooks.Open(excel_file_path)

            # Save the workbook as CSV, overwriting the original file
            workbook.SaveAs(csv_file_path, FileFormat=6)  # FileFormat=6 for CSV format

            # Close the workbook
            workbook.Close()

            # Delete the original file
            os.remove(excel_file_path)

    # Quit Excel
    excel.Quit()


In [3]:
#Fixing dates format


def convert_date_format(line):
    # Extract the date string from the line
    date_line = line.strip().split(',')[1:]
    
    for date in date_line:
        if date != "":
            # Convert the month name to its corresponding number
            day, month_name, year = date.split('-')
            month_number = str(list(calendar.month_abbr).index(month_name.capitalize()))

            # Format the date as 'dd/mm/yyyy'
            formatted_date = f"{day}/{month_number}/{year}"

            # Replace the original date string with the formatted date
            line = line.replace(date, formatted_date)

    return line

path = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Forecasts\\'

#Iterate over folders
#for item in ["GDP", "HCIP", "IndProd", "Unemployment"]:
for item in ["IndProd2"]:
    
    folder_path = os.path.join(path, item)

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):  # Process only CSV files
            csv_file_path = os.path.join(folder_path, filename)

            # Read the CSV file as a list of lines
            with open(csv_file_path, 'r') as file:
                lines = file.readlines()

            # Update the date format in specific lines (9 and 17)
            lines[8] = convert_date_format(lines[8])
            lines[16] = convert_date_format(lines[16])

            # Write the modified lines back to the CSV file, overwriting the original file
            with open(csv_file_path, 'w') as file:
                file.writelines(lines)

In [4]:
#Building dataset to run regressions

# Read the reference dates from source1.xlsx
source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'
df_source = pd.read_excel(source_file, header=0, sheet_name=3)

# Get the reference dates from the first column
reference_dates = df_source.iloc[:, 0]

# Create a DataFrame with the reference dates
result_df = pd.DataFrame(data=reference_dates)

# Add the quarter column
result_df['Quarter'] = pd.PeriodIndex(result_df['date'], freq='Q')

# Read the comp.csv file and get the fourth column
#csv_file = 'comp.csv'
#df_csv = pd.read_csv(csv_file, header=None)
#fourth_column = df_csv.iloc[8, :]

# Add the fourth column to the result DataFrame
#result_df['Fourth Column'] = fourth_column

path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Forecasts\\"
kk = 'p'

for j in range(-1,3):
    
    
    
    #Loop begins here
    #Taking the following n forecasts
    result_df[f"Quarter_{j if j >= 0 else 'p'}"] = result_df['Quarter'].apply(lambda x: x + j)
    #result_df['Next Quarter2'] = result_df['Quarter'].apply(lambda x: x + 2)

    result_df[f"Quarter_{j if j >= 0 else 'p'}"] = pd.PeriodIndex(result_df[f"Quarter_{j if j >= 0 else 'p'}"]).strftime('Q%q %Y')

    #Getting indexes where the quarters change
    idlist = result_df.index[result_df['Quarter'].ne(result_df['Quarter'].shift().bfill())].tolist()
    #Adding zero
    idlist.insert(0,0)
    
    for var in ['GDP', 'HCIP', 'Unemployment', 'IndProd2']:
        
        path = path1 + f"{var}\\"
        for i in idlist:


            # Print the resulting DataFrame
            csvfile_path = path + f"{result_df.loc[i,'Quarter_'+f'{j if j >= 0 else kk}']}.csv"

            if f"{result_df.loc[i,'Quarter_'+f'{j if j >= 0 else kk}']}.csv" in os.listdir(path):
                #Getting only the relevant info and transposing the dataframe
                csvfile = (pd.read_csv(csvfile_path, header=None, skiprows = lambda x: x not in [8,9], index_col=0).dropna(axis=1)).T


                #Converting first column to datetype data
                csvfile['Consensus'] = pd.to_datetime(csvfile['Consensus'], format="%d/%m/%Y")
                print(csvfile_path)
                # Function to find the closest date in the csvfile for each given date of result_df in the target quarter

                def find_closest_date(date):
                    aux = pd.DataFrame(csvfile['Consensus'].apply(lambda x: (x - date)))
                    indx = aux[aux['Consensus']<datetime.timedelta(days=0)].idxmax() 
                    
                    date_aux = csvfile.loc[indx,'Consensus'].squeeze()
                    #The value of the forecast at the given horizon
                    var_aux = float(csvfile.loc[indx,'Median'].squeeze())
                    
                    
                    #Cmputing second dataframe to get revisions
                    aux2 = aux[aux['Consensus']< aux.loc[indx, 'Consensus'].squeeze()]
                    
                    if aux2.empty:
                        revision_aux = math.nan
                    else:    
                        indx2 = aux2.idxmax() 
                        revision_aux = float(csvfile.loc[indx2,'Median'].squeeze())
                    
                    return date_aux, var_aux, revision_aux 

                #Selecting only the target quarter to be replaced in result_df

                auxres = result_df[result_df[f"Quarter_{j if j >= 0 else 'p'}"] == result_df.loc[i,f"Quarter_{j if j >= 0 else 'p'}"]]
                #aux1 = auxres['date']
                #aux1 = aux1.apply(find_closest_date)
                #result_df['Close'] = auxres['date'].apply(find_closest_date)

                #Unpacking date and the forecast value
                auxi = pd.DataFrame(auxres['date'].apply(find_closest_date).to_list(), index=auxres.index)

                #result_df.loc[auxres.index,f"Closest_date_{j if j >= 0 else 'p'}"] = auxi[0]
                result_df.loc[auxres.index,f"{var}_{j if j >= 0 else 'p'}"] = auxi[1]
                result_df.loc[auxres.index,f"{var}_Rev_{j if j >= 0 else 'p'}"] = auxi[1] - auxi[2]

        #auxres.loc[3, 'date'] = 3
        #print(auxi)
        #ak = find_closest_date(result_df['date'][0])
        #print(auxres)
        #print(ak)
        #print(auxres['date'])
        #print(csvfile)
        #print(auxres)
        #auxres.at[3,'date'] = find_closest_date(auxres['date'][3])
    #print(result_df.iloc[:,:])


G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Resea

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q2 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q3 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2003.csv
G:\Other computers\My laptop\Wor

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2002.csv
G:\Other computers\My laptop\Workarea\Re

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q2 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Resea

...

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 2023.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\GDP\Q4 2001.csv
G:\Other computers\My laptop\Workarea\

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q2 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q3 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q4 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q1 2003.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\HCIP\Q2 2003.csv
G:\Other computers\My laptop\Wor

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2000.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2001.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2002.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2002.csv
G:\Other computers\My laptop\Workarea\Re

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q3 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q4 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q1 2023.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\Unemployment\Q2 2023.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 1999.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 1999.csv
G:\Other computers\My laptop\Workarea\Research\T

G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 2020.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 2020.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q2 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 2021.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q1 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q2 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q3 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\Q4 2022.csv
G:\Other computers\My laptop\Workarea\Research\ThesisII\DATA\Forecasts\IndProd2\

In [55]:
result_df.iloc[67:,-6:].isnull().values.any()


False

In [3]:
#Now getting industrial production

#Adding the year column
result_df['Year'] = pd.PeriodIndex(result_df['date'], freq='Y')

#Getting indexes where the year changes
idlisty = result_df.index[result_df['Year'].ne(result_df['Year'].shift().bfill())].tolist()

#Adding zero
idlisty.insert(0,0)

#Path of the folder
path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Forecasts\\"

var = 'IndProd'

path = path1 + var
for j in range(-1,2):
    
    
    
    #Loop begins here
    #Taking the following n forecasts
    result_df[f"Year_{j if j >= 0 else 'p'}"] = result_df['Year'].apply(lambda x: x + j)
    #result_df['Next Quarter2'] = result_df['Quarter'].apply(lambda x: x + 2)

    #result_df[f"Year_{j if j >= 0 else 'p'}"] = pd.PeriodIndex(result_df[f"Quarter_{j if j >= 0 else 'p'}"]).strftime('%Y')

    for i in idlisty:
                
                # Print the resulting DataFrame
                csvfile_path = path + f"\\{result_df.loc[i,'Year_'+f'{j if j >= 0 else kk}']}.csv"
                
                if f"{result_df.loc[i,'Year_'+f'{j if j >= 0 else kk}']}.csv" in os.listdir(path):
                    #Getting only the relevant info and transposing the dataframe
                    csvfile = (pd.read_csv(csvfile_path, header=None, skiprows = lambda x: x not in [8,9], index_col=0).dropna(axis=1)).T
                    
                    #Converting first column to datetype data
                    csvfile['Consensus'] = pd.to_datetime(csvfile['Consensus'], format="%d/%m/%Y")

                    # Function to find the closest date in the csvfile for each given date of result_df in the target quarter

                    def find_closest_date(date):
                        aux = pd.DataFrame(csvfile['Consensus'].apply(lambda x: (x - date)))
                        aux1 = aux[aux['Consensus']<datetime.timedelta(days=0)]
                        
                        if aux1.empty:
                            date_aux = math.nan
                            var_aux = math.nan
                            revision_aux = math.nan
                        else:
                            indx = aux1.idxmax()
                            date_aux = csvfile.loc[indx,'Consensus'].squeeze()
                            #The value of the forecast at the given horizon
                            var_aux = float(csvfile.loc[indx,'Median'].squeeze())


                            #Cmputing second dataframe to get revisions
                            aux2 = aux[aux['Consensus']< aux.loc[indx, 'Consensus'].squeeze()]

                            if aux2.empty:
                                revision_aux = math.nan
                            else:    
                                indx2 = aux2.idxmax() 
                                revision_aux = float(csvfile.loc[indx2,'Median'].squeeze())

                        return date_aux, var_aux, revision_aux 

                    #Selecting only the target quarter to be replaced in result_df

                    auxres = result_df[result_df[f"Year_{j if j >= 0 else 'p'}"] == result_df.loc[i,f"Year_{j if j >= 0 else 'p'}"]]
                    #aux1 = auxres['date']
                    #aux1 = aux1.apply(find_closest_date)
                    #result_df['Close'] = auxres['date'].apply(find_closest_date)

                    #Unpacking date and the forecast value
                    auxi = pd.DataFrame(auxres['date'].apply(find_closest_date).to_list(), index=auxres.index)
                    
                  
                    #result_df.loc[auxres.index,f"Closest_date_{j if j >= 0 else 'p'}"] = auxi[0]
                    result_df.loc[auxres.index,f"{var}_{j if j >= 0 else 'p'}"] = auxi[1]
                    result_df.loc[auxres.index,f"{var}_Rev_{j if j >= 0 else 'p'}"] = auxi[1] - auxi[2]



In [6]:
variables = ['HCIP', 'Unemployment', 'IndProd2']
und = ['_', '_Rev_']
index = [0,1,2,'p']
forecasts = result_df[[f"{v}{u}{i}" for v,u,i in itertools.product(variables, und, index) if not ((v=='Unemployment') and ((i in [2, 1, 'p']) and (u != '_Rev_')))]]
forecasts.to_csv(path1+'forecasts.csv')

In [7]:
for v,u,i in itertools.product(variables, und, index): 
    if not ((v=='Unemployment') and ((i in [2, 1, 'p']) and (u != '_Rev_'))):
        print(f"{v}{u}{i}")

HCIP_0
HCIP_1
HCIP_2
HCIP_p
HCIP_Rev_0
HCIP_Rev_1
HCIP_Rev_2
HCIP_Rev_p
Unemployment_0
Unemployment_Rev_0
Unemployment_Rev_1
Unemployment_Rev_2
Unemployment_Rev_p
IndProd2_0
IndProd2_1
IndProd2_2
IndProd2_p
IndProd2_Rev_0
IndProd2_Rev_1
IndProd2_Rev_2
IndProd2_Rev_p


In [8]:
forecasts.iloc[60:].isnull().values.any()

False

# Getting instruments

###                                 Auxiliary section - skip it

In [6]:
import statsmodels.api as sm

In [10]:
#df_source = df_source.set_index(pd.Series(range(0,291)))
#forecasts = forecasts.set_index(pd.Series(range(0,291)))
y = df_source['OIS_3M'][71:]
x = forecasts.iloc[71:,:]
#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x, missing='drop').fit()

#view model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 OIS_3M   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.7909
Date:                Tue, 20 Jun 2023   Prob (F-statistic):              0.729
Time:                        04:04:42   Log-Likelihood:                -493.14
No. Observations:                 220   AIC:                             1030.
Df Residuals:                     198   BIC:                             1105.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  1.1135      1

In [394]:
path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\"
instrument = model.resid
# Get the range of indexes to include
#index_range = range(instrument.index.min(), instrument.index.max() + 1)


##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
# Reindex the DataFrame with the full index range

instrument = instrument.reindex(index=range(0,291))

#instrument.to_csv(path3+'instrument.csv')

In [21]:
forecasts.iloc[1,:]

HCIP_0                 1.2
HCIP_1                 1.4
HCIP_2                 1.2
HCIP_p                 NaN
HCIP_Rev_0             NaN
HCIP_Rev_1             NaN
HCIP_Rev_2             NaN
HCIP_Rev_p             NaN
Unemployment_0        11.4
Unemployment_Rev_0     NaN
Unemployment_Rev_1     NaN
Unemployment_Rev_2     NaN
Unemployment_Rev_p     NaN
IndProd2_0             5.9
IndProd2_1             4.1
IndProd2_2             3.6
IndProd2_p             NaN
IndProd2_Rev_0         NaN
IndProd2_Rev_1         NaN
IndProd2_Rev_2         NaN
IndProd2_Rev_p         NaN
Name: 1, dtype: float64

In [95]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [301]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]
  
print(vif_data)

               feature          VIF
0                const   155.239727
1               HCIP_0    30.228077
2               HCIP_1    19.527943
3               HCIP_p    10.862594
4           HCIP_Rev_0     6.616293
5           HCIP_Rev_1     6.537127
6           HCIP_Rev_p     1.518219
7       Unemployment_0  2594.611643
8       Unemployment_1  1170.689206
9       Unemployment_p   699.921644
10  Unemployment_Rev_0    30.298530
11  Unemployment_Rev_1    27.309692
12  Unemployment_Rev_p    11.683796
13           IndProd_0     8.555906
14           IndProd_p     3.318789
15       IndProd_Rev_0     4.958280
16       IndProd_Rev_p     3.941431


In [449]:
##THIS CELL IS NOT NECESSARY IF THE INSTRUMENT HAS NO GAPS

#Adding month mark
result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')

#Creating right dataframe to be converted into a month frequency
resid = pd.DataFrame(instrument, columns=['Values'])

#Adding the month column to the new dataframe
resid['Month'] = result_df.loc[resid['Values'].index, 'Month']

In [296]:
#Adding month mark
result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')

#Getting indexes when it changes
idlistm = result_df.index[result_df['Month'].ne(result_df['Month'].shift().bfill())].tolist()
idlistm.insert(0,0)

In [3]:
#This version do not put zeros in nans

# def sum_values_by_month(df):
#     # Convert the month column to datetime type
#     #df['Month'] = pd.to_datetime(df['Month'])

#     # Create a new dataframe with a complete range of months
#     min_month = df['Month'].min().to_timestamp()
#     max_month = df['Month'].max().to_timestamp()
#     complete_months = pd.date_range(start=min_month, end=max_month, freq='M')
#     complete_df = pd.DataFrame({'Month': complete_months})
    
#     complete_df['Month'] = pd.PeriodIndex(complete_df['Month'], freq='M')
#     # Merge the original dataframe with the complete dataframe to fill missing months with NaN values
#     merged_df = pd.merge(complete_df, df, on='Month', how='left')

#     # Group the merged dataframe by month and calculate the sum of values for each month
#     result = merged_df.groupby('Month')['Values'].sum().reset_index()

#     # Fill NaN values with 0
#     #result['Values'] = result['Values'].fillna(0)

#     return result


In [505]:
min_month = resid['Month'].min().to_timestamp()
max_month = resid['Month'].max().to_timestamp()
complete_months = pd.date_range(start=min_month, end=max_month, freq='M')
complete_df = pd.DataFrame({'Month': complete_months})
complete_df['Month'] = pd.PeriodIndex(complete_df['Month'], freq='M')

merged_df = pd.merge(complete_df, resid, on='Month', how='left')
result = merged_df.groupby('Month')['Values'].sum().reset_index()
resid[75:]

,Values,Month
150,-11.170705,2008-08
151,2.999867,2008-09
152,-5.868454,2008-10
153,3.231546,2008-10
154,13.180311,2008-11
...,...,...
263,3.062053,2019-06
264,1.831951,2019-07
265,7.497884,2019-09
266,-0.008584,2019-10


In [9]:
from statsmodels.tsa.ar_model import AutoReg

In [108]:
atreg13 = AutoReg(MPI['Values'], lags=12).fit()

In [477]:
realinstrument = atreg13.resid

realinstrument.to_csv(path3+'realinstrument.csv')


In [109]:
print(atreg13.summary())

                            AutoReg Model Results                             
Dep. Variable:                 Values   No. Observations:                  190
Model:                    AutoReg(12)   Log Likelihood                -313.452
Method:               Conditional MLE   S.D. of innovations              1.408
Date:                Tue, 30 May 2023   AIC                            654.904
Time:                        20:21:55   BIC                            699.449
Sample:                            12   HQIC                           672.968
                                  190                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0028      0.106      0.026      0.979      -0.204       0.210
Values.L1     -0.1158      0.075     -1.554      0.120      -0.262       0.030
Values.L2      0.0022      0.074      0.030      0.9

In [ ]:
###ORIGINAL LOOP

#Loop version
for k in ['SW', '1M', '3M', '6M', '1Y', '2Y']:

    y = df_source[f"OIS_{k}"][71+4:268]
    x = forecasts.iloc[71+4:268,:]
    #add constant to predictor variables
    x = sm.add_constant(x)

    #fit linear regression model
    model = sm.OLS(y, x, missing='drop').fit()


    path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\"
    instrument = model.resid
    # Get the range of indexes to include
    #index_range = range(instrument.index.min(), instrument.index.max() + 1)


    ##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
    # Reindex the DataFrame with the full index range
    instrument = instrument.reindex(index=range(0,291))

    instrument.to_csv(path3+f"instrument{k}.csv")
    
    resid = pd.DataFrame(instrument, columns=['Values'])
    #Adding month mark
    result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')
    
    #Adding the month column to the new dataframe
    resid['Month'] = result_df.loc[resid['Values'].index, 'Month']
    
    MPI = sum_values_by_month(resid)
    
    atreg12 = AutoReg(MPI['Values'], lags=12, missing='drop').fit()

    realinstrument = atreg12.resid

    realinstrument.to_csv(path3+f"realinstrument{k}.csv")


# Execute from here

In [3]:
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg

In [4]:
#Loading dataframes
source_file = 'G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\Instrument\\Dataset_EA-MPD.xlsx'
df_source = pd.read_excel(source_file, header=0, sheet_name=1)

path1 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Forecasts\\"
forecasts = pd.read_csv((path1+'forecasts.csv'), header=0, index_col=0)

# Get the reference dates from the first column
reference_dates = df_source.iloc[:, 0]

# Create a DataFrame with the reference dates
result_df = pd.DataFrame(data=reference_dates)

In [6]:
def sum_values_by_month(df):
    # Convert the month column to datetime type
    #df['Month'] = pd.to_datetime(df['Month'])
    
    #df = df.groupby('Month')['Values'].sum().reset_index()

    
    # Create a new dataframe with a complete range of months
    min_month = df['Month'].min().to_timestamp()
    max_month = df['Month'].max().to_timestamp() + datetime.timedelta(days=32)
    complete_months = pd.date_range(start=min_month, end=max_month, freq='M')
    complete_df = pd.DataFrame({'Month': complete_months})
    
    complete_df['Month'] = pd.PeriodIndex(complete_df['Month'], freq='M')
    # Merge the original dataframe with the complete dataframe to fill missing months with NaN values
    merged_df = pd.merge(complete_df, df, on='Month', how='left')

    # Group the merged dataframe by month and calculate the sum of values for each month
    #result = merged_df.groupby('Month')['Values'].sum().reset_index()
    
    result = merged_df
    # Fill NaN values with 0
    result['Values'] = result['Values'].fillna(0)

    return result

In [20]:
#Loop version

df_source = pd.read_excel(source_file, header=0, sheet_name=2)

# variables = ['HCIP', 'Unemployment', 'IndProd2']
# und = ['_', '_Rev_']
# index = [0,1,2,'p']
# forecasts = result_df[[f"{v}{u}{i}" for v,u,i in itertools.product(variables, und, index) if not ((v=='Unemployment') and ((i in [2, 1, 'p']) and (u != '_Rev_')))]]
# forecasts.to_csv(path1+'forecasts.csv')

#Period a = initial month/year index, b final month/year index
a = 70
b = 268 #No covid

for k in ['SW', '1M', '3M', '6M', '1Y', '2Y']:

    y = df_source[f"OIS_{k}"][a:]
    x = forecasts.iloc[a:,:]
    #add constant to predictor variables
    x = sm.add_constant(x)

    #fit linear regression model
    model = sm.OLS(y, x, missing='drop').fit()


    path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Instruments\\"
    instrument = model.resid
    # Get the range of indexes to include
    #index_range = range(instrument.index.min(), instrument.index.max() + 1)


    ##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
    # Reindex the DataFrame with the full index range
    #instrument = instrument.reindex(index=range(0,291))

    instrument.to_csv(path3+f"instrument{k}_autcor.csv")
    
    resid = pd.DataFrame(instrument, columns=['Values'])
    #Adding month mark
    result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')
    
    #Adding the month column to the new dataframe
    resid['Month'] = result_df.loc[resid['Values'].index, 'Month']
    
    MPI = resid.groupby('Month')['Values'].sum().reset_index()
    
    atreg12 = AutoReg(MPI['Values'], lags=12, missing='drop').fit()

    realinstrument =  pd.DataFrame(atreg12.resid, columns=['Values'])
    realinstrument['Month'] = MPI.loc[12:, 'Month']
    realinstrument = sum_values_by_month(realinstrument)
    realinstrument.to_csv(path3+f"instrument{k}.csv")

In [21]:
realinstrument

,Month,Values
0,2003-01,-0.757217
1,2003-02,0.637589
2,2003-03,-3.243570
3,2003-04,-1.507390
4,2003-05,4.474302
...,...,...
233,2022-06,0.644169
234,2022-07,-0.345111
235,2022-08,0.000000
236,2022-09,4.476672


# Be careful with this cell

In [38]:
#Pure instrument

#Loop version

#Period a = initial month/year index, b final month/year index
a = 75
#b = 268

for k in ['SW', '1M', '3M', '6M', '1Y', '2Y']:

    y = df_source[f"OIS_{k}"][a:]
    x = forecasts.iloc[a:,:]
    #add constant to predictor variables
    x = sm.add_constant(x)

    #fit linear regression model
    model = sm.OLS(y, x, missing='drop').fit()


    path3 = "G:\\Other computers\\My laptop\\Workarea\\Research\\ThesisII\\DATA\\Pure Instruments\\"
    instrument = y
    # Get the range of indexes to include
    #index_range = range(instrument.index.min(), instrument.index.max() + 1)


    ##CHECK THIS LINEEEE!!!! WHAT IS IT DOING?
    # Reindex the DataFrame with the full index range
    #instrument = instrument.reindex(index=range(0,291))

    instrument.to_csv(path3+f"instrument{k}.csv")
    
    resid = pd.DataFrame(instrument).rename(columns={f"OIS_{k}":'Values'})
    #Adding month mark
    result_df['Month'] = pd.PeriodIndex(result_df['date'], freq='M')
    
    #Adding the month column to the new dataframe
    resid['Month'] = result_df.loc[resid['Values'].index, 'Month']
    
    MPI = resid.groupby('Month')['Values'].sum().reset_index()
    
    #atreg12 = AutoReg(MPI['Values'], lags=12, missing='drop').fit()

    #realinstrument =  pd.DataFrame(atreg12.resid, columns=['Values'])
    #realinstrument['Month'] = MPI.loc[12:, 'Month']
    realinstrument = sum_values_by_month(MPI)
    realinstrument.to_csv(path3+f"realinstrument{k}.csv")

In [31]:
df_source.iloc[111:,:]

,date,OIS_SW,OIS_1M,OIS_3M,OIS_6M,OIS_1Y,OIS_2Y,OIS_3Y,OIS_4Y,OIS_5Y,...,ES5Y,FR5Y,ES10Y,FR10Y,IT10Y,STOXX50,SX7E,EURUSD,EURGBP,EURJPY
111,2005-05-04,-0.250000,-0.100000,0.000000,0.250000,0.350000,0.700000,0.500000,NaN,NaN,...,1.100000,1.500000,2.350000,2.400000,2.300000,0.255663,0.269804,-0.015440,0.058724,-0.014792
112,2005-06-02,1.100000,0.500000,0.200000,2.000000,2.200000,1.700000,1.600000,NaN,NaN,...,1.850000,1.850000,0.800000,0.800000,0.800000,-0.143928,-0.134437,-0.159148,-0.022208,-0.064051
113,2005-07-07,0.600000,0.500000,0.500000,2.100000,3.245000,4.200000,5.050000,NaN,NaN,...,4.950000,5.300000,3.650000,4.250000,4.350000,0.933381,0.733900,-0.560108,-0.576202,-0.231793
114,2005-08-04,0.000000,0.000000,0.000000,0.300000,0.000000,-0.200000,-0.150000,NaN,NaN,...,-0.300000,-0.300000,-0.150000,-0.200000,-0.250000,-0.052428,-0.015414,-0.077074,0.014415,-0.054588
115,2005-09-01,-0.500000,0.000000,0.000000,-0.300000,-1.400000,-2.650000,-2.050000,NaN,NaN,...,-3.100000,-2.900000,-2.100000,-2.300000,-2.050000,0.117994,0.151978,0.125023,-0.138965,0.072998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,2022-04-14,-0.500003,-0.500000,-1.790002,-3.899997,-6.360000,-6.720001,-4.699999,-3.699994,-2.600002,...,-2.249992,-2.590001,-0.250006,0.850010,-2.649999,0.236830,0.553515,-1.078603,-0.507191,-0.624157
287,2022-06-09,-2.200001,-0.974998,-4.189998,-1.220000,5.094999,8.864999,7.799995,7.450008,6.700003,...,15.249991,10.165000,11.949992,7.549989,18.200016,-1.410963,-2.042242,-0.289536,-0.339679,-0.052452
288,2022-07-21,11.899999,2.595000,14.669999,11.219996,8.999997,3.950000,1.400006,-1.249993,-3.450000,...,-1.699996,-0.514996,-3.050017,-4.699993,4.799986,0.119943,0.257638,0.147053,0.087728,-0.166328
289,2022-09-08,4.700002,6.415004,8.120000,10.200000,10.659993,7.249999,4.100001,3.600001,4.799986,...,9.050000,8.934999,7.200003,7.199979,6.949997,-0.223804,1.665142,-0.610582,-0.005766,-0.233269
